# 문제: 파일에서 RDD 생성
다음 링크에서 파일을 읽어서 RDD를 생성하고, 5줄을 화면출력하세요.

1) 경기도 의정부시 인구현황 (파일명: 경기도 의정부시_인구현황_20200904) https://www.data.go.kr/data/15009613/fileData.do

2) 제주특별자치도 서귀포시 내 연도별 65세이상 인구수 및 고령화비율, 노령화지수 현황 (파일명: 제주특별자치도 서귀포시_고령화비율및노령화지수현황_20200623) https://www.data.go.kr/data/15051545/fileData.do

## 주의:
SparkSession은 한 번만 생성하면 된다. 파일을 2번 읽는다 하더라도, SparkSession은 세션 동안 1회만 생성하면 된다.
파일을 읽을 경우, 문자가 한글인지, 영어인지 어떻게 인코딩되었는지 주의해야 한다. 결과가 깨져보인다면, 그 이유를 간단히 적으세요.

## SparkSession 생성

```py
import os
import pyspark

os.environ["PYSPARK_PYTHON"]="/usr/bin/python3" #파이썬 2,3 같이 깔린 경우
os.environ["PYSPARK_DRIVER_PYTHON"]="/usr/bin/python3" #파이썬 2,3 같이 깔린 경우
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate() #세션은 한번만 생성하면 된다.
```
난 이걸로 하면 계속 오류나서 아래걸로 하겠다.

In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

## RDD로 읽기

In [4]:
import os
popRdd = spark.sparkContext\
    .textFile(os.path.join("data","경기도 의정부시_인구현황_20200904.csv"), use_unicode=True)

In [5]:
type(popRdd)

pyspark.rdd.RDD

In [6]:
myRdd2=spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_wiki.txt"))

In [7]:
popRdd.take(5)

['�������,�α���(��),�α���(��),�α���(��),������(��),������(��),������(��),����,�����,������α�,���������,�����μ���,�μ���ȭ��ȣ,�����ͱ�������',
 '������1��       ,31467 ,16077 ,15390 ,6.89,3.52,3.37,104.46,19109 ,1.65,�����ν�,�ùκ����,031-828-2464,2020-09-04',
 '������2��       ,31440 ,15573 ,15867 ,6.88,3.41,3.47,98.15,15763 ,1.99,�����ν�,�ùκ����,031-828-2464,2020-09-04',
 'ȣ��1��         ,36282 ,17751 ,18531 ,7.95,3.89,4.06,95.79,15508 ,2.34,�����ν�,�ùκ����,031-828-2464,2020-09-04',
 'ȣ��2��         ,35443 ,17196 ,18247 ,7.76,3.77,4,94.24,13509 ,2.62,�����ν�,�ùκ����,031-828-2464,2020-09-04']

In [8]:

for i in popRdd.take(5):
    print(i)

�������,�α���(��),�α���(��),�α���(��),������(��),������(��),������(��),����,�����,������α�,���������,�����μ���,�μ���ȭ��ȣ,�����ͱ�������
������1��       ,31467 ,16077 ,15390 ,6.89,3.52,3.37,104.46,19109 ,1.65,�����ν�,�ùκ����,031-828-2464,2020-09-04
������2��       ,31440 ,15573 ,15867 ,6.88,3.41,3.47,98.15,15763 ,1.99,�����ν�,�ùκ����,031-828-2464,2020-09-04
ȣ��1��         ,36282 ,17751 ,18531 ,7.95,3.89,4.06,95.79,15508 ,2.34,�����ν�,�ùκ����,031-828-2464,2020-09-04
ȣ��2��         ,35443 ,17196 ,18247 ,7.76,3.77,4,94.24,13509 ,2.62,�����ν�,�ùκ����,031-828-2464,2020-09-04


화면에 출력하면 한글이 깨져있다. use_unicode=True설정을 주었는데도 그렇다. 다운로드 받으면서 한글이 깨져 있기 때문에 그렇다. 다운로드 받은 파일을 수정해서 출력하면 된다.

In [9]:
agedRdd = spark.sparkContext\
    .textFile(os.path.join("data","제주특별자치도 서귀포시_고령화비율및노령화지수현황_20200623.csv"), use_unicode=True)

In [10]:
for i in agedRdd.take(5):
    print(i)

������,�������� �α���,65���̻� �α��� ,0~14�� �α���,���ȭ����,���ȭ����,�����ͱ�������
2012,154057,25826,22861,16.76,112.97,2020-06-23
2013,155641,26936,22393,17.31,120.29,2020-06-23
2014,158512,27877,22058,17.59,126.38,2020-06-23
2015,164519,28979,22362,17.61,129.59,2020-06-23


## binaryFiles
binaryFiles()는 이진파일을 읽는 함수이다.

In [14]:
# RDD로 읽을때 spark.sparkContext 사용!!
popRddBin = spark.sparkContext.binaryFiles(os.path.join("data","경기도 의정부시_인구현황_20200904.csv"))

In [15]:
_my = popRddBin.map(lambda x :x[1].decode('euc-kr')) #[1]만 읽혀도 하나로되어있어서 다읽힌다

In [16]:
_my.take(1) # \r 한칸띄기

['행정기관,인구수(계),인구수(남),인구수(여),구성비(계),구성비(남),구성비(여),성비,세대수,세대당인구,관리기관명,관리부서명,부서전화번호,데이터기준일자\r\n의정부1동       ,31467 ,16077 ,15390 ,6.89,3.52,3.37,104.46,19109 ,1.65,의정부시,시민봉사과,031-828-2464,2020-09-04\r\n의정부2동       ,31440 ,15573 ,15867 ,6.88,3.41,3.47,98.15,15763 ,1.99,의정부시,시민봉사과,031-828-2464,2020-09-04\r\n호원1동         ,36282 ,17751 ,18531 ,7.95,3.89,4.06,95.79,15508 ,2.34,의정부시,시민봉사과,031-828-2464,2020-09-04\r\n호원2동         ,35443 ,17196 ,18247 ,7.76,3.77,4,94.24,13509 ,2.62,의정부시,시민봉사과,031-828-2464,2020-09-04\r\n장암동          ,20806 ,9935 ,10871 ,4.56,2.18,2.38,91.39,8585 ,2.42,의정부시,시민봉사과,031-828-2464,2020-09-04\r\n신곡1동         ,44492 ,21944 ,22548 ,9.74,4.81,4.94,97.32,18076 ,2.46,의정부시,시민봉사과,031-828-2464,2020-09-04\r\n신곡2동         ,44929 ,21764 ,23165 ,9.84,4.77,5.07,93.95,17491 ,2.57,의정부시,시민봉사과,031-828-2464,2020-09-04\r\n송산1동         ,36504 ,18113 ,18391 ,7.99,3.97,4.03,98.49,15830 ,2.31,의정부시,시민봉사과,031-828-2464,2020-09-04\r\n송산2동         ,33932 ,16782 ,17150 ,7.43,3.67,3.76,97.85,12991 ,2.6

RDD binaryFiles로 읽으니, 파일의 전체내용을 하나의 값으로 읽을 뿐만 아니라, 2차원 배열로 읽어도 행렬의 구조가 없어서 이해하기 어렵다. 인덱스를 변경해서 데이터를 읽어 보자.

In [17]:
popList = _my.map(lambda x: x.split()).take(3)
popList[0][1]

'의정부1동'

In [18]:
popList[0]

['행정기관,인구수(계),인구수(남),인구수(여),구성비(계),구성비(남),구성비(여),성비,세대수,세대당인구,관리기관명,관리부서명,부서전화번호,데이터기준일자',
 '의정부1동',
 ',31467',
 ',16077',
 ',15390',
 ',6.89,3.52,3.37,104.46,19109',
 ',1.65,의정부시,시민봉사과,031-828-2464,2020-09-04',
 '의정부2동',
 ',31440',
 ',15573',
 ',15867',
 ',6.88,3.41,3.47,98.15,15763',
 ',1.99,의정부시,시민봉사과,031-828-2464,2020-09-04',
 '호원1동',
 ',36282',
 ',17751',
 ',18531',
 ',7.95,3.89,4.06,95.79,15508',
 ',2.34,의정부시,시민봉사과,031-828-2464,2020-09-04',
 '호원2동',
 ',35443',
 ',17196',
 ',18247',
 ',7.76,3.77,4,94.24,13509',
 ',2.62,의정부시,시민봉사과,031-828-2464,2020-09-04',
 '장암동',
 ',20806',
 ',9935',
 ',10871',
 ',4.56,2.18,2.38,91.39,8585',
 ',2.42,의정부시,시민봉사과,031-828-2464,2020-09-04',
 '신곡1동',
 ',44492',
 ',21944',
 ',22548',
 ',9.74,4.81,4.94,97.32,18076',
 ',2.46,의정부시,시민봉사과,031-828-2464,2020-09-04',
 '신곡2동',
 ',44929',
 ',21764',
 ',23165',
 ',9.84,4.77,5.07,93.95,17491',
 ',2.57,의정부시,시민봉사과,031-828-2464,2020-09-04',
 '송산1동',
 ',36504',
 ',18113',
 ',18391',
 ',7.99,3.97,4.03,98.49,15830',
 ',2.

## DataFrame
데이터를 읽어서 DataFrame을 만드려면 read() 함수를 사용한다. 폰트를 변경하거나 option("charset", "euc-kr"), 헤더를 읽을지 option("header", "true") 설정을 변경할 수 있어 RDD보다 편리하다.

In [26]:
popDf = spark\
            .read.option("charset", "euc-kr")\
            .option("header", "true")\ #header 읽을건지?
            .csv(os.path.join("data","경기도 의정부시_인구현황_20200904.csv"))

In [27]:
popDf.show(5)

+----------------+----------+----------+----------+----------+----------+----------+------+------+----------+----------+----------+------------+--------------+
|        행정기관|인구수(계)|인구수(남)|인구수(여)|구성비(계)|구성비(남)|구성비(여)|  성비|세대수|세대당인구|관리기관명|관리부서명|부서전화번호|데이터기준일자|
+----------------+----------+----------+----------+----------+----------+----------+------+------+----------+----------+----------+------------+--------------+
|의정부1동       |    31467 |    16077 |    15390 |      6.89|      3.52|      3.37|104.46|19109 |      1.65|  의정부시|시민봉사과|031-828-2464|    2020-09-04|
|의정부2동       |    31440 |    15573 |    15867 |      6.88|      3.41|      3.47| 98.15|15763 |      1.99|  의정부시|시민봉사과|031-828-2464|    2020-09-04|
|호원1동         |    36282 |    17751 |    18531 |      7.95|      3.89|      4.06| 95.79|15508 |      2.34|  의정부시|시민봉사과|031-828-2464|    2020-09-04|
|호원2동         |    35443 |    17196 |    18247 |      7.76|      3.77|         4| 94.24|13509 |      2.62|  의정부시|시민봉사과|031-828-2464|    202

In [28]:
agedDf = spark\
            .read.option("charset", "euc-kr")\
            .option("header", "true")\
            .csv(os.path.join("data","제주특별자치도 서귀포시_고령화비율및노령화지수현황_20200623.csv"))

In [29]:
agedDf.show(5)

+------+---------------+----------------+-------------+----------+----------+--------------+
|연도별|서귀포시 인구수|65세이상 인구수 |0~14세 인구수|고령화비율|노령화지수|데이터기준일자|
+------+---------------+----------------+-------------+----------+----------+--------------+
|  2012|         154057|           25826|        22861|     16.76|    112.97|    2020-06-23|
|  2013|         155641|           26936|        22393|     17.31|    120.29|    2020-06-23|
|  2014|         158512|           27877|        22058|     17.59|    126.38|    2020-06-23|
|  2015|         164519|           28979|        22362|     17.61|    129.59|    2020-06-23|
|  2016|         170932|           30030|        23044|     17.57|    130.32|    2020-06-23|
+------+---------------+----------------+-------------+----------+----------+--------------+
only showing top 5 rows



## spark-submit 실행
위 프로그램을 .py로 저장하고, spark-submit 명령으로 배치 실행을 할 수 있다.
- 한꺼번에 일괄 실행

In [30]:
%%writefile src/ds3_popCsvRead.py
#!/usr/bin/env python3
# -*- coding: UTF-8 -*-
import os
import pyspark

def doIt():
    print ("---------RESULT-----------")
    popDf = spark\
                .read.option("charset", "euc-kr")\
                .option("header", "true")\
                .csv(os.path.join("data","경기도 의정부시_인구현황_20200904.csv"))
    popDf.show(5)
    agedDf = spark\
                .read.option("charset", "euc-kr")\
                .option("header", "true")\
                .csv(os.path.join("data","제주특별자치도 서귀포시_고령화비율및노령화지수현황_20200623.csv"))
    agedDf.show(5)

if __name__ == "__main__":
    os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"
    os.environ["PYSPARK_DRIVER_PYTHON"]="/usr/bin/python3"
    myConf=pyspark.SparkConf()
    spark = pyspark.sql.SparkSession.builder\
        .master("local")\
        .appName("myApp")\
        .config(conf=myConf)\
        .getOrCreate()
    doIt()
    spark.stop()

Overwriting src/ds3_popCsvRead.py


In [ ]:
!spark-submit src/ds3_popCsvRead.py
#오래걸린다

##